In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import torch
import torch.nn as nn
import torch.optim as optim
import os

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.models import resnet18,googlenet, densenet121,mobilenet_v3_small,mobilenet_v3_large
from sklearn.model_selection import train_test_split
from PIL import Image


In [8]:

def multiple_dir_files_to_df(dirs, label):
    df = pd.DataFrame()
    for idx,dir in enumerate(dirs):
        files = os.listdir(dir)
        files = [dir + '/' + file for file in files]
        df = pd.concat([df,pd.DataFrame({'file': files, 'label': label[idx]})])
    return df

df_au = pd.read_csv(f"../AU_models/dataset/Database.csv")

labels=["neutral","happy","sad","fear","angry","surprise","disgust"]
full_labels =["neutral","happy","sad","fear","angry","surprise","disgust","Happily_Surprised",
"Happily_Disgusted","Sadly_Fearful","Sadly_Angry","Sadly_Surprised","Sadly_Disgusted",
"Fearfully_Angry","Fearfully_Surprised","Fearfully_Disgusted","Angrily_Surprised",
"Angrily_Disgusted","Disgustedly_Surprised","Appalled","Hatred","Awed"]

# database=[common_path+label for label in labels]
# database_full=[common_path+label for label in full_labels]

def training_test_model(train_loader, val_loader, test_loader, model,model_name,epochs=50, learning_rate=0.001,weight_decay=1e-4,gamma=0.9,early_stopping_patience=5,early_stopping_save=True):
    # Initialiser le modèle et le journal
    model = model
    patience = early_stopping_patience
    best_val_loss = float('inf')
    counter = 0
    accuracy_val_list=[]
    accuracy_train_list=[]
    loss_val_list=[]
    loss_train_list=[]
    
    # Entraîner le modèle
    num_epochs = epochs
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Device: ",device)
    model.to(device)

    # Définir l'optimiseur et la fonction de perte
    optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer,gamma=gamma)
    # Entraîner le modèle
    for epoch in range(num_epochs):
        accuracy_train=[]
        loss_train=[]
        accuracy_val=[]
        loss_val=[]
        accuracy_test=[]
        loss_test=[]
        current_lr = [param_group['lr'] for param_group in optimizer.param_groups][0]

        model.train()
        for batch_idx, (data, targets, _) in enumerate(train_loader):
            # Get data to cuda if possible
            data = data.to(device=device)
            targets = targets.to(device=device)

            # forward
            scores = model(data)
            loss = criterion(scores, targets.long())

            # backward
            optimizer.zero_grad()
            loss.backward()

            optimizer.step()

            likelihood, predictions = scores.max(1)
            loss_train.append(loss.item())
            accuracy_train.append((predictions == targets).float().mean().item())

            # gradient descent or adam step
        scheduler.step()

        model.eval()
        for batch_idx, (data, targets, _) in enumerate(val_loader):
            # Get data to cuda if possible
            data = data.to(device=device)
            targets = targets.to(device=device)

            # forward
            scores = model(data)
            loss = criterion(scores, targets.long())
            
            likelihood, predictions = scores.max(1)
            loss_val.append(loss.item())
            accuracy_val.append((predictions == targets).float().mean().item())

        #testing the model
        for batch_idx, (data, targets, _)  in enumerate(test_loader):
            # Get data to cuda if possible
            data = data.to(device=device)
            targets = targets.to(device=device)

            # forward
            scores = model(data)
            loss = criterion(scores, targets.long())
            
            likelihood, predictions = scores.max(1)
            loss_test.append(loss.item())
            accuracy_test.append((predictions == targets).float().mean().item())

        accuracy_train=np.array(accuracy_train).mean()
        loss_train=np.array(loss_train).mean()
        accuracy_val=np.array(accuracy_val).mean()
        loss_val=np.array(loss_val).mean()
        accuracy_test=np.array(accuracy_test).mean()
        loss_test=np.array(loss_test).mean()
        accuracy_val_list.append(accuracy_val)
        accuracy_train_list.append(accuracy_train)
        loss_val_list.append(loss_val)
        loss_train_list.append(loss_train)
        if loss_val < best_val_loss:
                best_val_loss = loss_val
                best_val_accuracy = accuracy_val
                counter = 0
                best_params = model.state_dict()
                best_test_accuracy = accuracy_test
                best_val_accuracy = accuracy_val
        if early_stopping_save:
            if loss_val < best_val_loss:
                best_val_loss = loss_val
                best_val_accuracy = accuracy_val
                counter = 0
                best_params = model.state_dict()
            else:
                counter += 1
            if counter > patience:
                print("Early stopping at epoch: ",epoch)
                print("Saving model...")
                saved_params = model.state_dict()
                # get percentage of accuracy
                best_val_accuracy = np.round(best_val_accuracy*100,0)
                torch.save(saved_params, f"./models_tg/{model_name}_{str(int(best_val_accuracy))}.pth")
                return accuracy_val_list,accuracy_train_list,loss_val_list,loss_train_list
        print(f"Train epoch: {epoch}, accuracy = {accuracy_train} ,loss = {loss_train}, lr = {np.round(current_lr,6)}")
        print(f"val epoch: {epoch}, accuracy = {accuracy_val} ,loss = {loss_val}")
        print(f"test epoch: {epoch}, accuracy = {accuracy_test} ,loss = {loss_test}")

    torch.save(model.state_dict(), rf"./models_tg/{model_name}_{str(int(accuracy_val))}_{str(int(accuracy_test))}.pth")


    return accuracy_val_list,accuracy_train_list,loss_val_list,loss_train_list

def plot_train_loss(accuracy_test_list,accuracy_train_list,loss_test_list,loss_train_list,model_name):
    plt.figure(figsize=(10,5))
    plt.plot(accuracy_test_list,label="accuracy_test")
    plt.plot(accuracy_train_list,label="accuracy_train")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title("Accuracy for "+model_name+" model")
    plt.legend()
    plt.figure(figsize=(10,5))
    plt.plot(loss_test_list,label="loss_test")
    plt.plot(loss_train_list,label="loss_train")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Loss for "+model_name+" model")
    plt.legend()


# df = multiple_dir_files_to_df(database,np.linspace(0,6,7))
# X,y=df["file"].tolist(),df["label"].tolist()

# #Split the dataset into train and test
# df_full = multiple_dir_files_to_df(database_full,np.linspace(0,21,22))

df_full = pd.read_csv(f"database_train.csv")


X_full= ["database_train/" + s for s in df_full["filenames"].values]
y_full= [int(i.split("_")[0]) -1 for i in df_full["filenames"].values]





#get data with labels
X_train = [X_full[i] for i in range(len(X_full)) if int(y_full[i]) < 7]
y_train = [y_full[i] for i in range(len(y_full)) if int(y_full[i]) < 7]

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

print("Train head: ",X_train[:5],"\n",y_train[:5])
print("Test head: ",X_val[:5],"\n",y_val[:5])

# Définir un Dataset personnalisé
# Définir un Dataset personnalisé
class CustomDataset(Dataset):
    def __init__(self, file_names, labels,  transform=None):
        self.file_names = file_names
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        img_path = f'{self.file_names[idx]}' 
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        image, label = image.to(device), label.to(device)

        return image, label, self.file_names[idx]

# Define the image transformations
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
])
# Créer les jeux de données et les chargeurs de données
train_dataset = CustomDataset(X_train, y_train, transform=train_transform)
val_dataset = CustomDataset(X_val, y_val, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


class MobileNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.mobilenet = mobilenet_v3_small(weights='IMAGENET1K_V1')
        self.mobilenet.classifier[3] = nn.Linear(1024, 7)
    def forward(self, x):
        return self.mobilenet(x)

Train head:  ['database_train/06_081.jpg', 'database_train/07_222.jpg', 'database_train/06_209.jpg', 'database_train/03_073.jpg', 'database_train/07_160.jpg'] 
 [5, 6, 5, 2, 6]
Test head:  ['database_train/01_206.jpg', 'database_train/06_111.jpg', 'database_train/02_190.jpg', 'database_train/01_076.jpg', 'database_train/02_093.jpg'] 
 [0, 5, 1, 0, 1]


In [9]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [10]:
df_test = pd.read_csv(f"database_test.csv")


X_test_full= ["database_test/" + s for s in df_test["filenames"].values]
y_test_full= [int(i.split("_")[0]) -1 for i in df_test["filenames"].values]



#get data with labels
X_test = [X_test_full[i] for i in range(len(X_test_full)) if int(y_test_full[i]) < 7]
y_test = [y_test_full[i] for i in range(len(y_test_full)) if int(y_test_full[i]) < 7]

test_loader = DataLoader(CustomDataset(X_test, y_test, transform=test_transform), batch_size=32, shuffle=False)

In [11]:
model=MobileNet()
accuracy_val_list,accuracy_train_list,loss_val_list,loss_train_list=training_test_model(train_loader,val_loader, test_loader,model,"MobileNet",epochs=50, learning_rate=0.001,weight_decay=1e-4,gamma=0.9,early_stopping_patience=5,early_stopping_save=False)
plot_train_loss(accuracy_val_list,accuracy_train_list,loss_val_list,loss_train_list,"Mobile_Net")


Device:  cuda
Train epoch: 0, accuracy = 0.45886536770396763 ,loss = 1.3513596802949905, lr = 0.001
val epoch: 0, accuracy = 0.3472222222222222 ,loss = 2.232545084423489
test epoch: 0, accuracy = 0.29545454680919647 ,loss = 2.3814975221951804
Train epoch: 1, accuracy = 0.7113855282465616 ,loss = 0.7413101295630137, lr = 0.0009
val epoch: 1, accuracy = 0.2743055555555556 ,loss = 3.1139051384396024
test epoch: 1, accuracy = 0.264678031206131 ,loss = 2.4884703954060874
Train epoch: 2, accuracy = 0.804491486814287 ,loss = 0.5271010891430907, lr = 0.00081
val epoch: 2, accuracy = 0.5 ,loss = 2.5700203312767878
test epoch: 2, accuracy = 0.5492424269517263 ,loss = 2.3116746743520102
Train epoch: 3, accuracy = 0.8410618272092607 ,loss = 0.4124825182888243, lr = 0.000729
val epoch: 3, accuracy = 0.5590277777777778 ,loss = 1.4666970835791693
test epoch: 3, accuracy = 0.5539772709210714 ,loss = 1.3985259930292766
Train epoch: 4, accuracy = 0.8661514321962992 ,loss = 0.35563602132929695, lr = 0.00

AttributeError: 'numpy.float64' object has no attribute 'append'

In [114]:
# Créer les jeux de données et les chargeurs de données
t_dataset = CustomDataset(X_test_full, y_test_full, test_transform)
t_loader = DataLoader(t_dataset, batch_size=10, shuffle=False)

In [115]:
class Feature_Extractor(nn.Module):
    def __init__(self):
        super().__init__()
        self.mobilenet = mobilenet_v3_small(weights='IMAGENET1K_V1')
        self.mobilenet.classifier = nn.Identity()

    def forward(self, x):
        return self.mobilenet(x)
        


In [116]:
pretrained_model =rf"models_tg/MobileNet_88.pth"
model = Feature_Extractor()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.load_state_dict(torch.load(pretrained_model, map_location=device), strict=False)
model.to(device)
model.eval()
df=pd.DataFrame()

for i, (x, y) in enumerate(t_loader):
    x = x.to(device)
    y = y.to(device)
    with torch.no_grad():
        output = model(x)
        df=pd.concat([df,pd.DataFrame(output.cpu().numpy())])
        print(df.shape)

(10, 576)
(20, 576)
(30, 576)
(40, 576)
(50, 576)
(60, 576)
(70, 576)
(80, 576)
(90, 576)
(100, 576)
(110, 576)
(120, 576)
(130, 576)
(140, 576)
(150, 576)
(160, 576)
(170, 576)
(180, 576)
(190, 576)
(200, 576)
(210, 576)
(220, 576)
(230, 576)
(240, 576)
(250, 576)
(260, 576)
(270, 576)
(280, 576)
(290, 576)
(300, 576)
(310, 576)
(320, 576)
(330, 576)
(340, 576)
(350, 576)
(360, 576)
(370, 576)
(380, 576)
(390, 576)
(400, 576)
(410, 576)
(420, 576)
(430, 576)
(440, 576)
(450, 576)
(460, 576)
(470, 576)
(480, 576)
(490, 576)
(500, 576)
(506, 576)


In [127]:
au_df = pd.read_csv("../AU_models/dataset/partial_database.csv")

In [131]:
au_df["filename"] = X_full

In [135]:
au_test = au_df[au_df["filename"].isin(X_test_full)]

In [150]:
au_test.reset_index(drop=True, inplace=True)

In [164]:
df.reset_index(inplace=True, drop=True)
dataset = df
#normalize data
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
scaler = MinMaxScaler((0,1))

# scaler = StandardScaler()

dataset.iloc[:,:] = scaler.fit_transform(dataset.iloc[:,:])
dataset = pd.concat([dataset,au_test],axis=1)
dataset.reset_index(inplace=True, drop=True)
dataset["labels"]=[int(x.split("/")[-1].split("_")[0]) for x in dataset["filename"]]
dataset["filename"]=X_test_full

In [165]:
au_test

,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,AU15_r,AU17_r,AU20_r,AU23_r,AU25_r,AU26_r,AU45_r,filename
0,0.09,0.00,0.55,0.00,0.00,0.00,0.00,0.23,0.00,0.00,0.26,0.49,0.00,0.00,0.00,0.26,0.39,Image/neutral/01_009.jpg
1,0.73,0.49,0.25,0.00,0.36,0.00,0.00,0.22,0.47,0.21,0.00,0.44,0.18,0.55,0.20,0.61,0.18,Image/neutral/01_013.jpg
2,0.00,0.00,0.73,0.00,0.46,0.04,0.00,0.40,0.39,1.08,0.63,0.56,0.43,0.73,0.01,0.35,0.37,Image/neutral/01_024.jpg
3,0.00,0.02,0.66,0.00,0.15,0.04,0.54,0.05,0.08,0.12,0.80,1.46,0.34,0.29,0.00,0.18,0.08,Image/neutral/01_027.jpg
4,0.52,0.22,0.34,0.04,0.12,0.05,0.00,0.36,0.23,0.02,0.72,1.71,0.39,0.55,0.00,0.85,0.40,Image/neutral/01_030.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.80,0.41,0.27,0.24,0.00,0.14,0.00,0.29,0.00,0.00,0.16,0.47,0.00,0.00,1.99,2.80,0.22,Image/Awed/22_149.jpg
502,1.22,0.35,0.90,0.00,0.71,1.10,0.00,0.83,0.93,0.14,0.13,1.29,0.02,0.00,1.18,2.96,0.00,Image/Awed/22_173.jpg
503,1.10,0.87,0.79,0.69,0.00,0.36,0.00,0.65,0.19,0.00,0.45,1.13,0.06,0.09,2.13,2.89,0.21,Image/Awed/22_197.jpg
504,1.80,0.47,0.60,0.78,0.08,0.70,0.00,0.47,0.41,0.00,0.16,0.94,0.00,0.00,1.89,2.52,0.00,Image/Awed/22_219.jpg


In [167]:
dataset.to_csv("testing_database",index=False)

In [ ]:
df["label"] = y_test
df ["filename"] = X_test

In [2]:
aug_transform = transforms.Compose([
    # transforms.CenterCrop((800, 800)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [7]:
type(au_test_full)

pandas.core.frame.DataFrame

In [3]:

train_full_dataset = CustomDataset(X_train_full, y_train_full, au_train_full, transform=aug_transform)
train_full_loader = DataLoader(train_full_dataset, batch_size=32, shuffle=False)

for i, (x, y, au) in enumerate(train_full_loader):
    print(x.shape, y.shape, au.shape)
    # plt.imshow(x[0].permute(1, 2, 0))
    break

KeyError: 0

In [39]:
f_loader = DataLoader(CustomDataset(X, y, aug_transform), batch_size=31, shuffle=False)


In [30]:
import pandas as pd
df = pd.read_csv(f"../AU_models/dataset/Database.csv")


In [ ]:
df

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,0,0.0,0.98,1,0.120585,0.015412,-0.992583,-0.132393,0.050480,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,0,0.0,0.98,1,0.042829,0.085608,-0.995408,-0.098998,0.062438,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,3,0,0.0,0.98,1,0.137724,0.030648,-0.989996,-0.020681,0.038882,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0,0.0,0.77,1,0.092643,0.302859,-0.948522,-0.123163,0.099764,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0,0.0,0.98,1,0.034448,-0.012279,-0.999331,-0.155066,0.001050,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5055,5056,0,0.0,0.88,1,-0.105966,-0.058768,-0.992632,-0.271235,0.051710,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
5056,5057,0,0.0,0.88,1,-0.038780,-0.106725,-0.993532,-0.240089,-0.021937,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
5057,5058,0,0.0,0.98,1,-0.023173,0.014200,-0.999631,-0.131889,0.021436,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
5058,5059,0,0.0,0.98,1,-0.007678,-0.052453,-0.998594,-0.115005,-0.005229,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [3]:
df[" x_0"], df[" y_0"]

KeyError: ' x_0'

In [42]:
#get first value from t_loader

for (image, label) in list(enumerate(train_full_loader))[:1]:
    plt.imshow(image[0].permute(1, 2, 0))

TypeError: int() argument must be a string, a bytes-like object or a number, not 'Image'